In [ ]:
pip install transformers

     |████████████████████████████████| 1.4MB 6.0MB/s 
     |████████████████████████████████| 2.9MB 63.1MB/s 
     |████████████████████████████████| 890kB 69.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=3416bd99f0ce16473fe205c707428486d78c3700d53e87799d4b3c8aa4d5ceff
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle
from tqdm import tqdm
import torch

In [ ]:
training_files = pickle.load(open("/content/drive/MyDrive/CS6120/training_files.pkl","rb"))

In [40]:
concept_df= pickle.load(open("/content/drive/MyDrive/CS6120/concept_df.pkl","rb"))
relation_df= pickle.load(open("/content/drive/MyDrive/CS6120/relation_df.pkl","rb"))
text_df = pickle.load(open("/content/drive/MyDrive/CS6120/text_df.pkl","rb"))

In [ ]:
all_files = list(set(list(concept_df['file_name'])))

In [ ]:
test_files = [entry for entry in all_files if entry not in training_files ]

In [ ]:
from transformers import BertTokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained("/content/drive/MyDrive/CS6120/NCBI_BERT_pubmed_mimic_uncased_L-12_H-768_A-12")

In [ ]:
relation_model_pp = pickle.load(open("/content/drive/MyDrive/CS6120/relation_finetuned_seq_pp.pkl","rb"))

In [ ]:
relation_model_pte = pickle.load(open("/content/drive/MyDrive/CS6120/relation_finetuned_seq_pte.pkl","rb"))

In [ ]:
relation_model_ptr = pickle.load(open("/content/drive/MyDrive/CS6120/relation_finetuned_seq_pte.pkl","rb"))

In [ ]:
relation_model_pp = relation_model_pp.to('cuda')
relation_model_pte = relation_model_pte.to('cuda')
relation_model_ptr = relation_model_ptr.to('cuda')

In [ ]:
relation_model = relation_model.eval()

In [ ]:
relation_model_pp = relation_model_pp.eval()
relation_model_pte = relation_model_pte.eval()
relation_model_ptr = relation_model_ptr.eval()

In [ ]:
import gc

In [ ]:
def get_embeddings_for_nsp(first_sentence, second_sentence):
        encoding = bert_tokenizer(first_sentence, second_sentence, return_tensors='pt')['input_ids'][0]

        return encoding

In [17]:
get_embeddings_for_nsp("I like that.","")

tensor([ 101, 1045, 2066, 2008, 1012,  102,  102])

In [ ]:
sentence_map = dict()

#Treatment improves medical problem
sentence_map['TrIP'] = ["{0} improves {1}."]

#Treatment worsens medical problem
sentence_map['TrWP'] = ["{0} worsens {1}."]

#Treatment causes medical problem
sentence_map['TrCP'] = ["{0} causes {1}."]

#Treatment is administered for the medical problem
sentence_map['TrAP'] = ["{0} is administered for {1}."]

#Treatment is not administered because of medical problem
sentence_map['TrNAP'] = ["{0} is not administered due to {1}."]

#Treatment is not administered because of medical problem
sentence_map['NoREL'] = ["{0} and {1} has relation."]

#Test reveals medical problem
sentence_map['TeRP'] = ["{0} reveals {1}."]

#Test conducted to investigate medical problem
sentence_map['TeCP'] = ["{0} is conducted to investigate {1}."]

#Medical problem indicates medical problem
sentence_map['PIP'] = ["{1} indicates {0}."]

In [ ]:
device = torch.device("cuda")

In [ ]:
relation_model = relation_model.cuda()

In [ ]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


In [ ]:
device = torch.device("cuda")

In [ ]:
relation_model = relation_model.eval()

In [ ]:
len(all_problems)

15396

In [18]:
relation_list = list()
exist_rec_list = list()

all_relations_dict = {"treatment":['TrIP','TrWP','TrCP','TrAP','TrNAP','NoREL'], "test":['TeRP','TeCP','NoREL'], "problem":['PIP','NoREL']}

all_problems = concept_df[(concept_df['concept_type']=='problem') & (concept_df['file_name'].isin(training_files))]

for i in tqdm(range(16)):
    relation_list_in = list()
    for index,row in all_problems.iloc[i*1000:min((i+1)*1000,len(all_problems))].iterrows():
        all_other_entities = concept_df[(concept_df['file_name']==row['file_name'])&(concept_df['source']==row['source'])&(concept_df['line_number']==row['line_number'])&(concept_df['begin_word_num']!=row['begin_word_num'])]
        
        for entity_index,entity_row in all_other_entities.iterrows():
            
            key_to_check = row["source"]+"#"+row["file_name"]+"#"+str(row["line_number"])+"#"+str(entity_row["begin_word_num"])+"#"+str(row["begin_word_num"])
            
            if(key_to_check in exist_rec_list):
                continue
            else:
                exist_rec_list.append(row["source"]+"#"+row["file_name"]+"#"+str(row["line_number"])+"#"+str(row["begin_word_num"])+"#"+str(entity_row["begin_word_num"]))
            
            relation_df_record = relation_df[(relation_df['file_name']==row['file_name'])&(relation_df['source']==row['source'])&(relation_df['from_line_number']==row['line_number'])&(((relation_df['from_begin_word_num']==row['begin_word_num']) & (relation_df['to_begin_word_num']==entity_row['begin_word_num'])) |((relation_df['from_begin_word_num']==entity_row['begin_word_num']) & (relation_df['to_begin_word_num']==row['begin_word_num'])))]
            
            rel_dict = dict()

            if(len(relation_df_record)==0):
                relation = 'NoREL'
            else:
                relation = relation_df_record.iloc[0]["relation_type"]

            rel_dict['relation'] = relation
            
            #remaining_relations = [entry for entry in all_relations_dict[entity_row['concept_type']] if entry != relation]
            
            #relation_label_list.append(relation_label_dict[relation])
        
            #remaining_relations = [entry for entry in all_relations_dict[entity_row['concept_type']] if entry != relation]
            
            first_sentence = text_df[text_df['file_name']==row['file_name']].iloc[0]['text'].split("\n")[row['line_number']-1]
            
            first_sentence.replace(row['text'],"@problem")
            
            if(entity_row['concept_type']=='problem'):
                relation_model = relation_model_pp
                first_sentence.replace(entity_row['text'],"@related_problem")
            elif(entity_row['concept_type']=='test'):
                relation_model = relation_model_pte
                first_sentence.replace(entity_row['text'],"@test")
            else:
                relation_model = relation_model_ptr
                first_sentence.replace(entity_row['text'],"@treatment")

            encoding = bert_tokenizer.encode(first_sentence)
                      
            input_tensor = torch.tensor(pad_sequences([encoding],maxlen=512, dtype="long", 
                      value=0, truncating="post", padding="post")).long()

            input_tensor = input_tensor.to('cuda')

            with torch.no_grad():
              embedding = relation_model(input_tensor)
              
              op_tensor = embedding[1][12][0][0].to('cpu')

              
              rel_dict['embedding'] = op_tensor.detach().numpy()
              rel_dict['type'] = entity_row['concept_type']
              relation_list_in.append(rel_dict)

            '''

            for entry in sentence_map[relation]:
                second_sentence = entry.format(entity_row['text'],row['text'])
                
                encoding = get_embeddings_for_nsp(first_sentence,second_sentence)
                
                input_ids = pad_sequences([encoding],maxlen=512, dtype="long", 
                              value=0, truncating="post", padding="post")
                
                input_tensor = torch.tensor(input_ids).long()
                
                input_tensor = input_tensor.to('cuda')

                with torch.no_grad():
                  embedding = relation_model(input_tensor)
                  
                  op_tensor = embedding[1][12][0][0].to('cpu')
                  rel_dict = dict()
                  rel_dict['embedding'] = op_tensor.detach().numpy()
                  rel_dict['label'] = 1
                  rel_dict['relation'] = relation
                  relation_list_in.append(rel_dict)

            if(relation != 'NoREL'):     
              for no_relation in remaining_relations:
                  for entry in sentence_map[no_relation]:
                      second_sentence = entry.format(entity_row['text'],row['text'])

                      encoding = get_embeddings_for_nsp(first_sentence,second_sentence)
                      
                      input_tensor = torch.tensor(pad_sequences([encoding],maxlen=512, dtype="long", 
                                value=0, truncating="post", padding="post")).long()

                      input_tensor = input_tensor.to('cuda')

                      with torch.no_grad():
                        embedding = relation_model(input_tensor)
                        
                        op_tensor = embedding[1][12][0][0].to('cpu')

                        rel_dict = dict()
                        rel_dict['embedding'] = op_tensor.detach().numpy()
                        rel_dict['label'] = 0
                        rel_dict['relation'] = no_relation
                        relation_list_in.append(rel_dict)
              '''
    relation_list.extend(relation_list_in)

100%|██████████| 16/16 [11:31<00:00, 43.22s/it]


In [39]:
relation_df.columns

Index(['relation', 'embedding', 'type'], dtype='object')

In [43]:
relation_list_test = list()
exist_rec_list = list()

all_relations_dict = {"treatment":['TrIP','TrWP','TrCP','TrAP','TrNAP','NoREL'], "test":['TeRP','TeCP','NoREL'], "problem":['PIP','NoREL']}

all_problems = concept_df[(concept_df['concept_type']=='problem') & (concept_df['file_name'].isin(test_files))]

for index,row in all_problems.iterrows():
    all_other_entities = concept_df[(concept_df['file_name']==row['file_name'])&(concept_df['source']==row['source'])&(concept_df['line_number']==row['line_number'])&(concept_df['begin_word_num']!=row['begin_word_num'])]
    
    for entity_index,entity_row in all_other_entities.iterrows():
        
        key_to_check = row["source"]+"#"+row["file_name"]+"#"+str(row["line_number"])+"#"+str(entity_row["begin_word_num"])+"#"+str(row["begin_word_num"])
        
        if(key_to_check in exist_rec_list):
            continue
        else:
            exist_rec_list.append(row["source"]+"#"+row["file_name"]+"#"+str(row["line_number"])+"#"+str(row["begin_word_num"])+"#"+str(entity_row["begin_word_num"]))
        
        relation_df_record = relation_df[(relation_df['file_name']==row['file_name'])&(relation_df['source']==row['source'])&(relation_df['from_line_number']==row['line_number'])&(((relation_df['from_begin_word_num']==row['begin_word_num']) & (relation_df['to_begin_word_num']==entity_row['begin_word_num'])) |((relation_df['from_begin_word_num']==entity_row['begin_word_num']) & (relation_df['to_begin_word_num']==row['begin_word_num'])))]
        
        rel_dict = dict()

        if(len(relation_df_record)==0):
            relation = 'NoREL'
        else:
            relation = relation_df_record.iloc[0]["relation_type"]

        rel_dict['relation'] = relation
        
        #remaining_relations = [entry for entry in all_relations_dict[entity_row['concept_type']] if entry != relation]
        
        #relation_label_list.append(relation_label_dict[relation])
    
        #remaining_relations = [entry for entry in all_relations_dict[entity_row['concept_type']] if entry != relation]
        
        first_sentence = text_df[text_df['file_name']==row['file_name']].iloc[0]['text'].split("\n")[row['line_number']-1]
        
        first_sentence.replace(row['text'],"@problem")
        
        if(entity_row['concept_type']=='problem'):
            relation_model = relation_model_pp
            first_sentence.replace(entity_row['text'],"@related_problem")
        elif(entity_row['concept_type']=='test'):
            relation_model = relation_model_pte
            first_sentence.replace(entity_row['text'],"@test")
        else:
            relation_model = relation_model_ptr
            first_sentence.replace(entity_row['text'],"@treatment")

        encoding = bert_tokenizer.encode(first_sentence)
                  
        input_tensor = torch.tensor(pad_sequences([encoding],maxlen=512, dtype="long", 
                  value=0, truncating="post", padding="post")).long()

        input_tensor = input_tensor.to('cuda')

        with torch.no_grad():
          embedding = relation_model(input_tensor)
          
          op_tensor = embedding[1][12][0][0].to('cpu')

          
          rel_dict['embedding'] = op_tensor.detach().numpy()
          rel_dict['type'] = entity_row['concept_type']
          relation_list_test.append(rel_dict)


In [20]:
import pandas as pd

In [ ]:
t = torch.cuda.get_device_properties(0).total_memory
c = torch.cuda.memory_cached(0)
a = torch.cuda.memory_allocated(0)
f = c-a

/usr/local/lib/python3.6/dist-packages/torch/cuda/memory.py:346: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  FutureWarning)


In [21]:
relation_df = pd.DataFrame(relation_list)

In [44]:
relation_df_test = pd.DataFrame(relation_list_test)

In [23]:
relation_df_problem = relation_df[relation_df['type']=='problem']

In [27]:
X_embedding_pp = np.vstack(list(relation_df_problem["embedding"]))                                
y_relation_pp =list(relation_df_problem["relation"]) 

In [28]:
clf_relation_pp = LogisticRegression(random_state=0,solver="lbfgs",max_iter=10000).fit(X_embedding_pp, y_relation_pp)

In [29]:
clf_relation_pp.score(X_embedding_pp,y_relation_pp)

0.8632237067488866

In [51]:
relation_df_test_problem = relation_df_test[relation_df_test['type']=='problem']
X_embedding_pp_test = np.vstack(list(relation_df_test_problem["embedding"]))                                
y_relation_pp_test =list(relation_df_test_problem["relation"]) 

relation_df_test_test = relation_df_test[relation_df_test['type']=='test']
X_embedding_pte_test = np.vstack(list(relation_df_test_test["embedding"]))                                
y_relation_pte_test =list(relation_df_test_test["relation"]) 

relation_df_test_treatment = relation_df_test[relation_df_test['type']=='treatment']
X_embedding_ptr_test = np.vstack(list(relation_df_test_treatment["embedding"]))                                
y_relation_ptr_test =list(relation_df_test_treatment["relation"]) 


In [46]:
clf_relation_pp.score(X_embedding_pp_test,y_relation_pp_test)

0.8414111440817672

In [47]:
from sklearn.metrics import f1_score

In [50]:
f1_score(y_relation_pp_test,clf_relation_pp.predict(X_embedding_pp_test),labels=['NoREL','PIP'], average=None)

array([0.91182401, 0.21276596])

In [61]:
precision_score(y_relation_pp_test,clf_relation_pp.predict(X_embedding_pp_test),labels=['NoREL','PIP'], average=None)

array([0.85906736, 0.47101449])

In [62]:
recall_score(y_relation_pp_test,clf_relation_pp.predict(X_embedding_pp_test),labels=['NoREL','PIP'], average=None)

array([0.97148437, 0.13742072])

In [54]:
labels_test = list(set(y_relation_pte_test))

In [55]:
labels_test

['NoREL', 'TeCP', 'TeRP']

In [56]:
f1_score(y_relation_pte_test,clf_relation_pte.predict(X_embedding_pte_test),labels=labels_test, average=None)

array([0.61553398, 0.26027397, 0.73969072])

In [63]:
precision_score(y_relation_pte_test,clf_relation_pte.predict(X_embedding_pte_test),labels=labels_test, average=None)

array([0.67161017, 0.48717949, 0.67291911])

In [64]:
recall_score(y_relation_pte_test,clf_relation_pte.predict(X_embedding_pte_test),labels=labels_test, average=None)

array([0.56810036, 0.17757009, 0.8211731 ])

In [57]:
labels_treatment = list(set(y_relation_ptr_test))

In [58]:
labels_treatment

['TrCP', 'TrIP', 'TrAP', 'NoREL', 'TrWP', 'TrNAP']

In [59]:
f1_score(y_relation_ptr_test,clf_relation_ptr.predict(X_embedding_ptr_test),labels=labels_treatment, average=None)

array([0.22047244, 0.07692308, 0.62944984, 0.76618883, 0.        ,
       0.        ])

In [65]:
precision_score(y_relation_ptr_test,clf_relation_ptr.predict(X_embedding_ptr_test),labels=labels_treatment, average=None)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([0.60869565, 1.        , 0.63458401, 0.70135747, 0.        ,
       0.        ])

In [66]:
recall_score(y_relation_ptr_test,clf_relation_ptr.predict(X_embedding_ptr_test),labels=labels_treatment, average=None)

array([0.13461538, 0.04      , 0.62439807, 0.84422658, 0.        ,
       0.        ])

In [ ]:
relation_df_treatment = relation_df[relation_df['type']=='test']

X_embedding_pte = np.vstack(list(relation_df_test["embedding"]))                                
y_relation_pte =list(relation_df_test["relation"])

clf_relation_pte = LogisticRegression(random_state=0,solver="lbfgs",max_iter=10000).fit(X_embedding_pte, y_relation_pte)


In [32]:
clf_relation_pte.score(X_embedding_pte,y_relation_pte)

0.7257564003103181

In [35]:
relation_df_treatment = relation_df[relation_df['type']=='treatment']

X_embedding_ptr = np.vstack(list(relation_df_treatment["embedding"]))                                
y_relation_ptr =list(relation_df_treatment["relation"])

clf_relation_ptr = LogisticRegression(random_state=0,solver="lbfgs",max_iter=10000).fit(X_embedding_ptr, y_relation_ptr)


In [34]:
clf_relation_ptr.score(X_embedding_ptr,y_relation_ptr)

0.7257564003103181

In [ ]:
relation_df_5 = pd.DataFrame(relation_list)

In [ ]:
relation_df_6 = pd.DataFrame(relation_list)

In [ ]:
del relation_model
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
pickle.dump(pd.DataFrame(relation_list),open("/content/drive/MyDrive/CS6120/rel_data_df.pkl","wb"))

In [ ]:
import pandas as pd

In [ ]:
import h5py

In [ ]:
hf.close()

In [ ]:
hf = h5py.File('/content/drive/MyDrive/CS6120/relation1.h5', 'w')

#store = pd.HDFStore('/content/drive/MyDrive/CS6120/relation1.h5')

#relation['df'] = pd.DataFrame(relation_list)  # save it
#relation['df']

In [ ]:
all_rel_data = pickle.load(open("/content/drive/MyDrive/CS6120/rel_data_df.pkl","rb"))

In [ ]:
all_rel_data = pd.DataFrame(relation_list)

In [ ]:
all_rel_data.columns

Index(['relation', 'embedding'], dtype='object')

In [ ]:
for index,row in all_rel_data.iterrows():
  if(row['relation'] in ['PNP','TeNRP','TrNRP']):
    all_rel_data.at[index,'relation'] = 'NoREL'

In [ ]:
all_rel_data.groupby(['relation','label']).size()

relation  label
NoREL     0         7307
          1        15899
PIP       1         1730
TeCP      0         2352
          1          397
TeRP      0          397
          1         2352
TrAP      0          835
          1         1993
TrCP      0         2406
          1          422
TrIP      0         2650
          1          178
TrNAP     0         2695
          1          133
TrWP      0         2726
          1          102
dtype: int64

In [ ]:
final_dataframe = all_rel_data[all_rel_data['relation']=='NoREL']

In [ ]:
for relation in labels:
  print(relation)
  if(relation=='NoREL'):
    filt_df_1 = all_rel_data[(all_rel_data['relation']=='NoREL')&(all_rel_data['label']==1)]
    filt_df_0 = all_rel_data[(all_rel_data['relation']=='NoREL')&(all_rel_data['label']==0)]

    sample_df_1 = filt_df_1.sample(len(filt_df_0))

    df_comb = pd.concat([sample_df_1,filt_df_0])
  else:
    df_comb = all_rel_data[all_rel_data['relation']==relation]
  
  if(len(final_dataframe)==0):
    final_dataframe = df_comb
  else:
    final_dataframe = pd.concat([final_dataframe,df_comb])


TrIP
TrWP
TrCP
TrAP
TrNAP
NoREL
TeRP
TeCP
PIP


In [ ]:
len(all_rel_data[all_rel_data['label']==1])

31037

In [24]:
from sklearn.linear_model import LogisticRegression

In [26]:
import numpy as np

In [ ]:
X_embedding = np.vstack(list(all_rel_data["embedding"]))                                
y_relation =list(all_rel_data["relation"]) 

In [ ]:
clf_relation = LogisticRegression(random_state=0,solver="lbfgs",max_iter=10000).fit(X_embedding, y_relation)

In [ ]:
clf_relation.predict_proba([all_rel_data.iloc[0]['embedding']])[0][1]

0.21110379453444092

In [ ]:
clf_relation.score(X_embedding, y_relation)

0.7340343014737568

In [ ]:
y_true = y_relation
y_predicted = clf_relation.predict(X_embedding)
f1_score(y_true,y_predicted,average=None)

array([0.84061123, 0.2070922 , 0.10502283, 0.43560606, 0.51303075,
       0.13419913, 0.07407407, 0.        , 0.        ])

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
list(set(y_relation))

['TrAP', 'PIP', 'TrCP', 'TrIP', 'TrWP', 'NoREL', 'TeRP', 'TeCP', 'TrNAP']

In [ ]:
confusion_matrix(y_relation,clf_relation.predict(X_embedding),list(set(y_relation)))

array([[  876,     5,     0,     0,     0,  1092,    16,     4,     0],
       [   42,   219,     3,     0,     0,  1380,    86,     0,     0],
       [   58,     8,    31,     0,     0,   322,     3,     0,     0],
       [   26,     1,     0,     7,     0,   142,     2,     0,     0],
       [   17,     0,     1,     1,     0,    82,     1,     0,     0],
       [  294,   129,     4,     1,     0, 15073,   389,     9,     0],
       [   34,    13,     0,     0,     0,  1495,   805,     5,     0],
       [   57,     8,     0,     2,     0,   267,    40,    23,     0],
       [   18,     2,     1,     0,     0,   110,     2,     0,     0]])

In [ ]:
all_problems = concept_df[(concept_df['concept_type']=='problem') & (concept_df['file_name'].isin(test_files))]
exist_rec_list = list()

all_relations_dict = {"treatment":['TrIP','TrWP','TrCP','TrAP','TrNAP','NoREL'], "test":['TeRP','TeCP','NoREL'], "problem":['PIP','NoREL']}

print(len(all_problems))

prediction_list = []

for index,row in tqdm(all_problems.iterrows()):
    all_other_entities = concept_df[(concept_df['file_name']==row['file_name'])&(concept_df['source']==row['source'])&(concept_df['line_number']==row['line_number'])&(concept_df['begin_word_num']!=row['begin_word_num'])]
    
    for entity_index,entity_row in all_other_entities.iterrows():
        
        key_to_check = row["source"]+"#"+row["file_name"]+"#"+str(row["line_number"])+"#"+str(entity_row["begin_word_num"])+"#"+str(row["begin_word_num"])
        
        if(key_to_check in exist_rec_list):
            continue
        else:
            exist_rec_list.append(row["source"]+"#"+row["file_name"]+"#"+str(row["line_number"])+"#"+str(row["begin_word_num"])+"#"+str(entity_row["begin_word_num"]))
        
        relation_df_record = relation_df[(relation_df['file_name']==row['file_name'])&(relation_df['source']==row['source'])&(relation_df['from_line_number']==row['line_number'])&(((relation_df['from_begin_word_num']==row['begin_word_num']) & (relation_df['to_begin_word_num']==entity_row['begin_word_num'])) |((relation_df['from_begin_word_num']==entity_row['begin_word_num']) & (relation_df['to_begin_word_num']==row['begin_word_num'])))]
        
        entry_dict = dict()

        if(len(relation_df_record)==0):
            entry_dict["True_label"] = 0
            relation = "NoREL"
        else:
            relation = relation_df_record.iloc[0]["relation_type"]
            entry_dict["True_label"] = 1

        entry_dict["True_relation"] = relation

        all_relations = [entry for entry in all_relations_dict[entity_row['concept_type']]]

        first_sentence = text_df[text_df['file_name']==row['file_name']].iloc[0]['text'].split("\n")[row['line_number']-1]
        
        entry = sentence_map['NoREL'][0]
        second_sentence = entry.format(entity_row['text'],row['text'])
        
        encoding = get_embeddings_for_nsp(first_sentence,second_sentence)

        input_tensor = torch.tensor(pad_sequences([encoding],maxlen=512, dtype="long", 
                  value=0, truncating="post", padding="post")).long()

        input_tensor = input_tensor.to('cuda')

        with torch.no_grad():
          embedding = relation_model(input_tensor)
          
          op_12 = embedding[0][0].to('cpu').detach().numpy()

          if(op_12[1]>op_12[0]):
            entry_dict["predicted_label"] = 1
          else:
            entry_dict["predicted_label"] = 0
          prediction_list.append(entry_dict)
        '''
        max_probability = 0
        predicted_relation = ""        

        for present_relation in all_relations:
            for entry in sentence_map[present_relation]:
                second_sentence = entry.format(entity_row['text'],row['text'])
                
                encoding = get_embeddings_for_nsp(first_sentence,second_sentence)

                input_tensor = torch.tensor(pad_sequences([encoding],maxlen=512, dtype="long", 
                          value=0, truncating="post", padding="post")).long()

                input_tensor = input_tensor.to('cuda')

                with torch.no_grad():
                  embedding = relation_model(input_tensor)
                  
                  op_tensor = embedding[1][12][0][0].to('cpu')

                  embedding_input = op_tensor.detach().numpy()

                  if(clf_relation.predict([embedding_input])[0]==1):
                    op_probab = clf_relation.predict_proba([embedding_input])[0][1]

                    if(op_probab>max_probability):
                      if(present_relation=="NoREL"):
                        if(predicted_relation==""):
                          predicted_relation = "NoREL"
                      else:
                        predicted_relation = present_relation
        
          
        if(predicted_relation == ""):
          predicted_relation = "NoREL"
        entry_dict["predicted_relation"] = predicted_relation
        prediction_list.append(entry_dict)
        '''

0it [00:00, ?it/s]

4269


4269it [03:11, 22.35it/s]


In [ ]:
encoding = get_embeddings_for_nsp("I am a boy.","No new thing.")

input_tensor = torch.tensor(pad_sequences([encoding],maxlen=512, dtype="long", 
          value=0, truncating="post", padding="post")).long()

input_tensor = input_tensor.to('cuda')

with torch.no_grad():
  embedding = relation_model(input_tensor)
  
  op_tensor = embedding[1][12][0][0].to('cpu')

  embedding_input = op_tensor.detach().numpy()

  #temp_op  = clf_relation.predict([embedding_input])[0]

In [ ]:
embedding[0][0].to('cpu').detach().numpy()[0]

3.3855474

In [ ]:
import pandas as pd


In [ ]:
pred_df = pd.DataFrame(prediction_list)

In [60]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [ ]:
y_true = list(pred_df['True_label'])
y_predicted = list(pred_df['predicted_label'])

In [ ]:
y_predicted[0:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
f1_score(y_true,y_predicted)

NameError: ignored

In [ ]:
accuracy_score(y_true,y_predicted)

0.6574360645056198

In [ ]:
labels = ['TrIP','TrWP','TrCP','TrAP','TrNAP','NoREL','TeRP','TeCP','PIP']

In [ ]:
confusion_matrix(y_true,y_predicted,labels)

array([[   0,    0,    0,    8,    0,   17,    0,    0,    0],
       [   0,    0,    0,   12,    0,   19,    0,    0,    0],
       [   0,    0,    2,   47,    0,   55,    0,    0,    0],
       [   0,    0,    1,  185,   20,  417,    0,    0,    0],
       [   0,    0,    0,    9,    3,   29,    0,    0,    0],
       [   0,    0,    5,  462,  195,  321,  367,  141, 2545],
       [   0,    0,    0,    0,    0,   71,  558,   70,    0],
       [   0,    0,    0,    0,    0,   21,   77,    9,    0],
       [   0,    0,    0,    0,    0,   10,    0,    0,  463]])

In [ ]:
len([entry for entry in y_true if entry=='TrWP'])

31